<a href="https://colab.research.google.com/github/buntys2010/AV-AmExpert-2019-ML-Hackathon/blob/master/Amexpert_2019_Part_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Get Data

In [0]:
from google.colab import files

In [0]:
files.upload()

In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle datasets download buntyshah/amexpert-2019

In [0]:
!unzip "../content/amexpert-2019.zip" 

## Read Data

In [0]:
import pandas as pd
import numpy as np
#  Libraries
import numpy as np 
import pandas as pd 
# Data processing, metrics and modeling
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold,KFold

from datetime import datetime
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, roc_curve, auc,precision_recall_curve
from sklearn import metrics
from sklearn import preprocessing
# Lgbm
import lightgbm as lgb
# Suppr warning
import warnings
warnings.filterwarnings("ignore")

import itertools
from scipy import interp

# Plots
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams

In [0]:
train= pd.read_csv("../content/train_AUpWtIz (1)/train.csv")

In [0]:
train.redemption_status.value_counts()

0    77640
1      729
Name: redemption_status, dtype: int64

In [0]:
test = pd.read_csv("../content/test_QyjYwdj.csv")

In [0]:
# Read 5 common files

campaign_data = pd.read_csv("../content/train_AUpWtIz (1)/campaign_data.csv")
coupon_item_mapping =  pd.read_csv("../content/train_AUpWtIz (1)/coupon_item_mapping.csv")
customer_demographics = pd.read_csv("../content/train_AUpWtIz (1)/customer_demographics.csv")
customer_transaction_data = pd.read_csv("../content/train_AUpWtIz (1)/customer_transaction_data.csv")
item_data = pd.read_csv("../content/train_AUpWtIz (1)/item_data.csv")

![alt text](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/09/Screenshot-2019-09-28-at-8.58.32-PM.png)

## Lets Combine campaign_id with Train/test as it doesnt have any dependency

In [0]:
train_df = pd.merge(train,campaign_data,on='campaign_id',how='left')
test_df = pd.merge(test,campaign_data,on='campaign_id',how='left')

## Lets Combine customer_demographics with train and test

In [0]:
train_df = pd.merge(train_df , customer_demographics ,on='customer_id',how='left')
test_df = pd.merge(test_df , customer_demographics ,on='customer_id',how='left')

## Create new DF to combine remaining blocks

Note : 



1.   Tried Many features combination of catagorical Features
2.   Tried to create many new features few worked as below



In [0]:
df = pd.merge(customer_transaction_data , item_data ,on='item_id',how='left')
df = pd.merge(df , coupon_item_mapping ,on='item_id',how='left')

In [0]:
df = df.fillna(120)

In [0]:
df['date'] = df['date'].astype('datetime64[ns]') 

In [0]:
df['C_date_year'] = df['date'].dt.year
df['C_date_month'] = df['date'].dt.month
df['C_date_day'] = df['date'].dt.day
df = df.drop('date',axis=1)

In [0]:
df.head()

,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category,coupon_id,C_date_year,C_date_month,C_date_day
0,1501,26830,1,35.26,-10.69,0.0,56,Local,Natural Products,7.0,2012,1,2
1,1501,26830,1,35.26,-10.69,0.0,56,Local,Natural Products,20.0,2012,1,2
2,1501,26830,1,35.26,-10.69,0.0,56,Local,Natural Products,29.0,2012,1,2
3,1501,54253,1,53.43,-13.89,0.0,56,Local,Natural Products,7.0,2012,1,2
4,1501,54253,1,53.43,-13.89,0.0,56,Local,Natural Products,20.0,2012,1,2


In [0]:
## Feature engg
df['per_item_price'] = df['selling_price'] / df['quantity']
df['item_price_after_discount'] = (df['selling_price'] + df['other_discount'] + df['coupon_discount']) / df['quantity']
df['discount'] = df['other_discount'] + df['coupon_discount']

In [0]:
df['NF_coupon_month'] = df['coupon_id'].astype(str) + "_" + df['C_date_month'].astype(str)

In [0]:
from sklearn.preprocessing import LabelEncoder
for f in df.columns:
    if  df[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df[f].values))
        df[f] = lbl.transform(list(df[f].values))

## Creating Aggrigate features for *customer_id*

Note: Tried to hide few features as their importance was almost Zero

In [0]:
['mean','sum','min','max','median','quantile','mad','std','count','nunique']
ag_cat=['count','nunique','min','max','last']
ag_num=['mean','sum','min','max','median','quantile','mad','std']

agg_func = {     # find the sum of the durations for each group
                                    'item_id' : ag_cat,
                                    'quantity':ag_num,
                                    'selling_price': ag_num,
                                    'other_discount':ag_num,
                                    #'coupon_discount':ag_num,
                                    'brand':ag_cat,
                                    'brand_type':ag_cat,
                                    'category':ag_cat,
                                    'coupon_id':ag_cat,
                                    #'C_date_year':ag_cat,
                                    'C_date_month':ag_cat,
                                    'C_date_day':ag_cat,
                                    #'per_item_price':ag_num,
                                    #'item_price_after_discount':ag_num,
                                    'NF_coupon_month':ag_cat
                                   
                                }

In [0]:
df_customer_groupby = df.groupby(by=['customer_id']).agg(agg_func)
df_customer_groupby.columns=['agg_cust_id_' + '_'.join(col).strip() for col in df_customer_groupby.columns.values]
df_customer_groupby.reset_index(inplace=True)
df_customer_groupby.head()

,customer_id,agg_cust_id_item_id_count,agg_cust_id_item_id_nunique,agg_cust_id_item_id_min,agg_cust_id_item_id_max,agg_cust_id_item_id_last,agg_cust_id_quantity_mean,agg_cust_id_quantity_sum,agg_cust_id_quantity_min,agg_cust_id_quantity_max,agg_cust_id_quantity_median,agg_cust_id_quantity_quantile,agg_cust_id_quantity_mad,agg_cust_id_quantity_std,agg_cust_id_selling_price_mean,agg_cust_id_selling_price_sum,agg_cust_id_selling_price_min,agg_cust_id_selling_price_max,agg_cust_id_selling_price_median,agg_cust_id_selling_price_quantile,agg_cust_id_selling_price_mad,agg_cust_id_selling_price_std,agg_cust_id_other_discount_mean,agg_cust_id_other_discount_sum,agg_cust_id_other_discount_min,agg_cust_id_other_discount_max,agg_cust_id_other_discount_median,agg_cust_id_other_discount_quantile,agg_cust_id_other_discount_mad,agg_cust_id_other_discount_std,agg_cust_id_brand_count,agg_cust_id_brand_nunique,agg_cust_id_brand_min,agg_cust_id_brand_max,agg_cust_id_brand_last,agg_cust_id_brand_type_count,agg_cust_id_brand_type_nunique,agg_cust_id_brand_type_min,agg_cust_id_brand_type_max,agg_cust_id_brand_type_last,agg_cust_id_category_count,agg_cust_id_category_nunique,agg_cust_id_category_min,agg_cust_id_category_max,agg_cust_id_category_last,agg_cust_id_coupon_id_count,agg_cust_id_coupon_id_nunique,agg_cust_id_coupon_id_min,agg_cust_id_coupon_id_max,agg_cust_id_coupon_id_last,agg_cust_id_C_date_month_count,agg_cust_id_C_date_month_nunique,agg_cust_id_C_date_month_min,agg_cust_id_C_date_month_max,agg_cust_id_C_date_month_last,agg_cust_id_C_date_day_count,agg_cust_id_C_date_day_nunique,agg_cust_id_C_date_day_min,agg_cust_id_C_date_day_max,agg_cust_id_C_date_day_last,agg_cust_id_NF_coupon_month_count,agg_cust_id_NF_coupon_month_nunique,agg_cust_id_NF_coupon_month_min,agg_cust_id_NF_coupon_month_max,agg_cust_id_NF_coupon_month_last
0,1,2211,463,4315,73348,45986,1.134328,2508,1,5,1,1.0,0.240588,0.454791,95.070873,210201.70,12.11,445.25,89.05,89.05,35.683296,53.433822,-20.346404,-44985.90,-372.23,0.0,-10.69,-10.69,21.518545,36.210123,2211,163,1,5403,1075,2211,2,0,1,0,2211,9,1,13,6,2211,160,1.0,1089.0,30.0,2211,12,1,12,6,2211,28,1,31,30,2211,725,1,12096,3726
1,2,823,352,137,73242,73242,1.117861,920,1,5,1,1.0,0.213669,0.417919,103.368943,85072.64,14.25,640.80,89.05,89.05,45.778112,71.076889,-16.798141,-13824.87,-312.03,0.0,0.00,0.00,20.244751,33.011891,823,153,1,5131,158,823,2,0,1,0,823,9,1,16,11,823,138,6.0,1109.0,32.0,823,11,2,12,6,823,19,1,28,26,823,302,138,12208,3970
2,3,1351,406,4308,73698,73698,6.861584,9270,1,6949,1,1.0,10.358366,189.016329,113.680792,153582.75,8.90,862.00,89.05,89.05,59.687796,85.572038,-24.400081,-32964.51,-463.06,0.0,-9.26,-9.26,25.908501,40.760823,1351,114,1,5220,172,1351,2,0,1,0,1351,8,4,16,6,1351,136,6.0,1109.0,120.0,1351,12,1,12,6,1351,23,1,31,16,1351,408,35,12171,1582
3,4,451,125,4617,65281,61047,1.283814,579,1,5,1,1.0,0.440509,0.624805,143.631774,64777.93,14.25,1330.05,106.86,106.86,72.674528,147.361484,-15.565854,-7020.20,-390.04,0.0,0.00,0.00,18.483995,31.132399,451,72,1,5303,5303,451,2,0,1,0,451,8,2,16,11,451,67,6.0,1098.0,32.0,451,11,1,12,5,451,23,1,30,11,451,207,284,11714,3969
4,5,1594,490,4314,71182,69320,59.345671,94597,1,14638,1,1.0,115.211085,826.140617,128.931066,205516.12,12.11,1485.35,99.38,99.38,77.382276,121.436279,-13.463024,-21460.06,-216.21,0.0,0.00,0.00,18.705007,28.304603,1594,168,1,5423,673,1594,2,0,1,0,1594,14,1,16,6,1594,122,6.0,1101.0,668.0,1594,12,1,12,6,1594,31,1,31,29,1594,412,289,12091,8237


In [0]:
train_df = pd.merge(train_df , df_customer_groupby ,on='customer_id',how='left')
test_df = pd.merge(test_df , df_customer_groupby ,on='customer_id',how='left')

## Creating Aggrigate features for coupon_id

In [0]:
['mean','sum','min','max','median','quantile','mad','std','count','nunique']
ag_cat=['count','nunique','min','max','last']
ag_num=['mean','sum','min','max','median','quantile','mad','std']

agg_func_1 = {     # find the sum of the durations for each group
                                    'customer_id' : ag_cat,
                                    'quantity':ag_num,
                                    'selling_price': ag_num,
                                    'other_discount':ag_num,
                                    'coupon_discount':ag_num,
                                    'brand':ag_cat,
                                    'brand_type':ag_cat,
                                    'category':ag_cat,
                                    'item_id':ag_cat,
                                    #'C_date_year':ag_cat,
                                    'C_date_month':ag_cat,
                                    'C_date_day':ag_cat,
                                    'per_item_price':ag_num,
                                    'item_price_after_discount':ag_num,
                                    'NF_coupon_month':ag_cat
                                }

In [0]:
df_coupon_id_groupby = df.groupby(by=['coupon_id']).agg(agg_func_1)
df_coupon_id_groupby.columns=['agg_cp_id_' + '_'.join(col).strip() for col in df_coupon_id_groupby.columns.values]
df_coupon_id_groupby.reset_index(inplace=True)
train_df = pd.merge(train_df , df_coupon_id_groupby ,on='coupon_id',how='left')
test_df = pd.merge(test_df , df_coupon_id_groupby ,on='coupon_id',how='left')

## Aggrigate with customer_id and coupon_id combine

Note : Creating this features provided boost from AUC of 89 to 92 in LB

In [0]:
['mean','sum','min','max','median','quantile','mad','std','count','nunique']
ag_cat=['count','nunique','min','max','last']
ag_num=['mean','sum','min','max','median','quantile','mad','std']

agg_func_2 = {     # find the sum of the durations for each group
                                   # 'customer_id' : ag_cat,
                                    'quantity':ag_num,
                                    'selling_price': ag_num,
                                    'other_discount':ag_num,
                                    'coupon_discount':ag_num,
                                    'brand':ag_cat,
                                    'brand_type':ag_cat,
                                    'category':ag_cat,
                                    'item_id':ag_cat,
                                    #'C_date_year':ag_cat,
                                    'C_date_month':ag_cat,
                                    'C_date_day':ag_cat,
                                    'per_item_price':ag_num,
                                    'item_price_after_discount':ag_num,
                                    'NF_coupon_month':ag_cat
                                }

In [0]:
df_cust_cp_id_groupby = df.groupby(by=['customer_id','coupon_id']).agg(agg_func_2)

In [0]:
df_cust_cp_id_groupby.columns=['agg_cust_cp_id_' + '_'.join(col).strip() for col in df_cust_cp_id_groupby.columns.values]
df_cust_cp_id_groupby.reset_index(inplace=True)
train_df = pd.merge(train_df , df_cust_cp_id_groupby ,on=['customer_id','coupon_id'],how='left')
test_df = pd.merge(test_df , df_cust_cp_id_groupby ,on=['customer_id','coupon_id'],how='left')

## Changes in Train and Test DF

In [0]:
train_df['start_date'] = pd.to_datetime(train_df['start_date'])
test_df['start_date'] = pd.to_datetime(test_df['start_date'])

train_df['end_date'] = pd.to_datetime(train_df['end_date'])
test_df['end_date'] = pd.to_datetime(test_df['end_date'])

In [0]:
#train_df['start_date_year'] = train_df['start_date'].dt.year
train_df['start_date_month'] = train_df['start_date'].dt.month
train_df['start_date_day'] = train_df['start_date'].dt.day

#test_df['start_date_year'] = test_df['start_date'].dt.year
test_df['start_date_month'] = test_df['start_date'].dt.month
test_df['start_date_day'] = test_df['start_date'].dt.day

In [0]:
#train_df['end_date_year'] = train_df['end_date'].dt.year
train_df['end_date_month'] = train_df['end_date'].dt.month
train_df['end_date_day'] = train_df['end_date'].dt.day

#test_df['end_date_year'] = test_df['end_date'].dt.year
test_df['end_date_month'] = test_df['end_date'].dt.month
test_df['end_date_day'] = test_df['end_date'].dt.day

In [0]:
train_df['campaign_run'] = (train_df['end_date']  - train_df['start_date'])/np.timedelta64(1,'D')
test_df['campaign_run'] = (test_df['end_date']  - test_df['start_date'])/ np.timedelta64(1,'D')

In [0]:
train_df = train_df.drop(['start_date','end_date','no_of_children','id'],axis=1)
test_df = test_df.drop(['start_date','end_date','no_of_children','id',],axis=1)

In [0]:
for f in train_df.columns:
    if  train_df[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_df[f].values))
        train_df[f] = lbl.transform(list(train_df[f].values))

In [0]:
for f in test_df.columns:
    if  test_df[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(test_df[f].values))
        test_df[f] = lbl.transform(list(test_df[f].values))

In [0]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [0]:
# Missing values statistics of Customers
missing_values = missing_values_table(train_df)
missing_values.head(10)

In [0]:
# Missing values statistics of Customers
missing_values = missing_values_table(test_df)
missing_values.head(10)

## Pre-Process

In [0]:
# Align train and test
train_labels = train_df['redemption_status']

# Align the training and testing data, keep only columns present in both dataframes
train_df, test_df = train_df.align(test_df, join = 'inner', axis = 1)

# Add the target back in
train_df['redemption_status'] = train_labels

print('Training Features shape: ', train_df.shape)
print('Testing Features shape: ', test_df.shape)

In [0]:
y = train_df['redemption_status']
X = train_df.drop('redemption_status',axis=1)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y.values,random_state=10)

## LGB

Note : LGB gave a good score of 92.7 on LB , After many attemps the score remained the same

In [0]:
def run_lgb(X_train, X_test, y_train, y_test, test_df):
  params = {
        "objective" : "binary",
       "n_estimators":10000,
        "learning_rate" :0.007,
        "num_leaves" :90,
        "min_data_in_leaf":3,
        'max_bin':200,
        'max_depth':3,
        "metric" : "auc",
        "reg_alpha" :3,
        "reg_lambda":5,
        'feature_fraction':0.8999999999999121,
        'min_gain_to_split':0.2,
        'bagging_fraction':0.8999999999997461,
        'bagging_freq':50,
        'scale_pos_weight':15.0
        
    }
    
  lgtrain = lgb.Dataset(X_train, label=y_train)
  lgval = lgb.Dataset(X_test, label=y_test)
  evals_result = {}
  model = lgb.train(params, lgtrain, 10000, 
                      valid_sets=[lgtrain, lgval], 
                      early_stopping_rounds=50, 
                      verbose_eval=50, 
                      evals_result=evals_result)
    
  pred_test_y = model.predict(test_df, num_iteration=model.best_iteration)
  return pred_test_y, model, evals_result

In [0]:
pred_test, model, evals_result = run_lgb(X_train, X_test, y_train, y_test, test_df)
print("LightGBM Training Completed...")

In [0]:
feature_imp = pd.DataFrame(sorted(zip(model.feature_importance(),X_train.columns)), columns=['Value','Feature'])
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(model.feature_importance(importance_type='gain'),X_train.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False).head(50))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()

In [0]:
sub = pd.read_csv('../content/Submission.csv')

In [0]:
sub['redemption_status'] = pred_test

In [0]:
sub.to_csv('LGB_Mode_Sub11.csv',index=False)

## Kfold

Note : Kfold provided better score then LGB

In [0]:
params = {
        "objective" : "binary",
       "n_estimators":10000,
        "learning_rate" :0.007,
        "num_leaves" :90,
        "min_data_in_leaf":3,
        'max_bin':200,
        'max_depth':3,
        "metric" : "auc",
        "reg_alpha" :3,
        "reg_lambda":5,
        'feature_fraction':0.8999999999999121,
        'min_gain_to_split':0.2,
        'bagging_fraction':0.8999999999997461,
        'bagging_freq':50,
        'scale_pos_weight':15.0
        
    }

In [0]:
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
errcb=[]
y_pred_totcb=[]
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=6,shuffle=True,random_state=1994)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lgtrain = lgb.Dataset(X_train, label=y_train)
    lgval = lgb.Dataset(X_test, label=y_test)
    model = lgb.train(params, lgtrain, 10000, 
                      valid_sets=[lgtrain, lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=100)
    preds = model.predict(X_test, num_iteration=model.best_iteration)
    print("err: ",roc_auc_score(y_test,preds))
    errcb.append(roc_auc_score(y_test,preds))
    p = model.predict(test_df, num_iteration=model.best_iteration)
    y_pred_totcb.append(p)

In [0]:
feature_imp = pd.DataFrame(sorted(zip(model.feature_importance(),X_train.columns)), columns=['Value','Feature'])
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(model.feature_importance(importance_type='gain'),X_train.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False).head(50))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()

In [0]:
y_pred = np.mean(y_pred_totcb,0)

In [0]:
sub['redemption_status']= y_pred

In [0]:
sub.to_csv("LGB_Kfold_Sub4.csv",index=False)

## Cat Booster

Note : Used skopt to find best learning parameter for CatBooster

In [0]:
!pip install catboost

In [0]:
from catboost import CatBoostClassifier,Pool, cv

In [0]:
import catboost

class ModelOptimizer:
    best_score = None
    opt = None
    
    def __init__(self, model, X_train, y_train, categorical_columns_indices=None, n_fold=3, seed=1994, early_stopping_rounds=30, is_stratified=True, is_shuffle=True):
        self.model = model
        self.X_train = X_train
        self.y_train = y_train
        self.categorical_columns_indices = categorical_columns_indices
        self.n_fold = n_fold
        self.seed = seed
        self.early_stopping_rounds = early_stopping_rounds
        self.is_stratified = is_stratified
        self.is_shuffle = is_shuffle
        
        
    def update_model(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self.model, k, v)
            
    def evaluate_model(self):
        pass
    
    def optimize(self, param_space, max_evals=10, n_random_starts=2):
        start_time = time.time()
        
        @use_named_args(param_space)
        def _minimize(**params):
            self.model.set_params(**params)
            return self.evaluate_model()
        
        opt = gp_minimize(_minimize, param_space, n_calls=max_evals, n_random_starts=n_random_starts, random_state=2405, n_jobs=-1)
        best_values = opt.x
        optimal_values = dict(zip([param.name for param in param_space], best_values))
        best_score = opt.fun
        self.best_score = best_score
        self.opt = opt
        
        print('optimal_parameters: {}\noptimal score: {}\noptimization time: {}'.format(optimal_values, best_score, time.time() - start_time))
        print('updating model with optimal values')
        self.update_model(**optimal_values)
        plot_convergence(opt)
        return optimal_values
    
class CatboostOptimizer(ModelOptimizer):
    def evaluate_model(self):
        validation_scores = catboost.cv(
        catboost.Pool(self.X_train, 
                      self.y_train, 
                      cat_features=self.categorical_columns_indices),
        self.model.get_params(), 
        nfold=self.n_fold,
        stratified=self.is_stratified,
        seed=self.seed,
        early_stopping_rounds=self.early_stopping_rounds,
        shuffle=self.is_shuffle,
        verbose=100,
        plot=False)
        self.scores = validation_scores
        test_scores = validation_scores.iloc[:, 2]
        best_metric = test_scores.max()
        return 1 - best_metric

In [0]:
!pip install scikit-optimize

In [0]:
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from skopt.plots import plot_convergence
import time

In [0]:
cb = catboost.CatBoostClassifier(n_estimators=4000, # use large n_estimators deliberately to make use of the early stopping
                         loss_function='Logloss',
                         eval_metric='AUC',
                         boosting_type='Ordered', # use permutations
                         random_seed=1994, 
                         use_best_model=True)
cb_optimizer = CatboostOptimizer(cb, X_train, y_train)
params_space = [Real(0.01, 0.8, name='learning_rate'),]
cb_optimal_values = cb_optimizer.optimize(params_space)

## Catbooster

Note : Catbooster provided highest score of 93.11 on LB with same parameters

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
m=CatBoostClassifier(n_estimators=3000,random_state=1994,eval_metric='AUC',max_depth=10,learning_rate=0.11389587749465084,od_wait=50
                     ,l2_leaf_reg=1,bagging_temperature=0.85,random_strength=100,
                     use_best_model=True)
m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
p2=m.predict_proba(test_df)[:,-1]

In [0]:
p2

In [0]:
sub['redemption_status'] = p2

In [0]:
sub.to_csv('CatBooster_all_features.csv',index=False)

In [0]:
sorted(zip(m.feature_importances_,X_train),reverse=True)

In [0]:
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
errcb=[]
y_pred_totcb=[]
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=3,shuffle=True,random_state=1994)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m=CatBoostClassifier(n_estimators=3000,random_state=1994,eval_metric='AUC',max_depth=10,learning_rate=0.11389587749465084,od_wait=50
                     ,l2_leaf_reg=1,bagging_temperature=0.85,random_strength=100,
                     use_best_model=True)
    m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m.predict_proba(X_test)[:,-1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(test_df)[:,-1]
    y_pred_totcb.append(p)

In [0]:
np.mean(errcb)

In [0]:
y_pred = np.mean(y_pred_totcb,0)

In [0]:
sub['redemption_status'] = y_pred

In [0]:
sub.to_csv('CatBooster_3Kfold_sun3.csv',index=False)

In [0]:
sorted(zip(m.feature_importances_,X_train),reverse=True)